In [1]:
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
def g(url):
    
    with requests.Session() as s:
        retries = Retry(total=10, backoff_factor=1, status_forcelist=[500, 502, 503, 504])

        s.mount('http://', HTTPAdapter(max_retries=retries))
        s.mount('https://', HTTPAdapter(max_retries=retries))

        return s.get(url, timeout=10)

In [3]:
sqlstr = lambda string: "'" + str(string).replace("'", "''") + "'"

### Results

In [4]:
## SQL create statements
stmts_faculty = [] # done
stmts_degree = []
stmts_subject = [] # done
stmts_course = [] # done
stmts_courseGroup = []
stmts_courseGroupMember = []
stmts_prerequisite = []
stmts_degreeRequirement = []
stmts_term = [] # done
stmts_courseOffering = [] # done

In [5]:
## Entity Sets for Later Use
set_faculty = set()
set_subject = set()
set_course = set()
set_courseOffering = set()

### Term

In [6]:
## strings for terms
terms = [str(100 + y) + str(t) for y in range (10, 21) for t in [1, 5, 9]]

In [7]:
stmts_term = [f'INSERT INTO term (code) VALUES ({term});' for term in terms]

### Faculty

In [8]:
with open('faculty.json', 'r') as faculty_f:
    faculties = json.load(faculty_f)

In [9]:
set_faculty = set()
stmts_faculty = []

for faculty in faculties:
    title = faculty['group_code']
    short_name = faculty['group_short_name']
    full_name = faculty['group_full_name']
    
    set_faculty.add(title)
    stmts_faculty.append(
        "INSERT INTO faculty (title, shortName, fullName) VALUES (%s, %s, %s);" % 
        (sqlstr(title), sqlstr(short_name), sqlstr(full_name))
    )

### Subject

In [10]:
with open('subject.json', 'r') as subject_f:
  subjects = json.load(subject_f)

In [11]:
set_subject = set()
stmts_subject = []

for subject in subjects:
    title = subject['subject']
    desc = subject['description']
    faculty = subject['group']
    
    if faculty in set_faculty:
        set_subject.add(title)
        stmts_subject.append(
            "INSERT INTO subject (title, facultyTitle, description) VALUES (%s, %s, %s);" % 
            (sqlstr(title), sqlstr(faculty), sqlstr(desc))
        )

### Courses

#### You can import the courses from `course.txt` if the file exists.

In [14]:
set_course = set()
with open('course.txt', 'r') as f:
    for line in f:
        set_course.add(line.strip())

#### Alternatively, you can scrape the course data from the OpenAPI as well.

In [15]:
courses_r = g('https://api.uwaterloo.ca/v2/courses.json?key=e5c5ea41b7715202b2846b04b286f29d')
courses = list(filter(lambda c: '`' not in c['catalog_number'], courses_r.json()['data'])) # there is a "GER 744`"

In [17]:
set_course = set()
stmts_course = []

l = len(courses)

for i, c in enumerate(courses):
    subject_title = c['subject']
    catalog_number = c['catalog_number']
    
    course_code = subject_title + ' ' + catalog_number
    
    ## print the progress of processing
    print('%s/%s course(s) processed. Processing %s.\t\t' % (i, l, course_code), end='\r', flush=True)

    course_detail_r = g(
        'https://api.uwaterloo.ca/v2/courses/%s/%s.json?key=e5c5ea41b7715202b2846b04b286f29d' %
        (subject_title, catalog_number)
    ).json()

    if 'data' in course_detail_r and course_detail_r['data'] and subject_title in set_subject:
        course_detail = course_detail_r['data']

        title = course_detail['title']
        credit = course_detail['units']
        desc = course_detail['description']
        course_types = ', '.join(ins for ins in course_detail['instructions'])

        set_course.add(course_code)
        stmts_course.append(
            "INSERT INTO course (courseCode, title, credit, courseTypes, description, subjectTitle) VALUES (%s, %s, %s, %s, %s, %s);" %
            (sqlstr(course_code),
             sqlstr(title),
             credit,
             sqlstr(course_types),
             sqlstr(desc),
             sqlstr(subject_title))
        )
        
print('\ndone.')

KeyboardInterrupt: 

### Course Offerings

In [ ]:
set_courseOffering = set()
stmts_courseOffering = []

l = len(set_subject)
l_terms = len(terms)

for i, subject_title in enumerate(set_subject):
    
    for j, term in enumerate(terms):
        
        ## print the progress of processing
        print('%s/%s terms out of %s/%s subject(s) processed. Processing term=%s, subject=%s. So far %s offering(s) recorded.\t\t' %
              (j, l_terms, i, l, term, subject_title, len(stmts_courseOffering)), end='\r', flush=True)
            
        offerings_r = g(
            'https://api.uwaterloo.ca/v2/terms/%s/%s/schedule.json?key=e5c5ea41b7715202b2846b04b286f29d' %
            (term, subject_title)
        ).json()
                
        if 'data' in offerings_r and offerings_r['data']:
            offerings = offerings_r['data']

            for offering in offerings:
                catalog_number = offering['catalog_number']
                course_code = subject_title + ' ' + catalog_number

                if course_code in set_course:
                    args = []
                    vals = []

                    args.append('courseCode')
                    vals.append(sqlstr(course_code))

                    args.append('termCode')
                    vals.append(term)

                    section = offering['section'].split(' ')

                    courseType = section[0]
                    component = section[1]
                    
                    if (course_code, term, component) not in set_courseOffering:
                        args.append('courseType')
                        vals.append(sqlstr(courseType))

                        args.append('component')
                        vals.append(component)

                        if offering['related_component_1']:
                            args.append('relatedComponent1')
                            vals.append(str(offering['related_component_1']))

                        if offering['related_component_2']:
                            args.append('relatedComponent2')
                            vals.append(str(offering['related_component_2']))

                        enrlCap = offering['enrollment_capacity']
                        enrlTot = offering['enrollment_total']

                        args.append('enrlCap')
                        vals.append(str(enrlCap))

                        args.append('enrlTot')
                        vals.append(str(enrlTot))

                        classes = offering['classes']
                        if classes:
                            cls = classes[0]

                            instructors = cls['instructors']
                            if instructors:
                                prof = instructors[0].split(',')

                                if len(prof) > 1:
                                    args.append('profFirstName')
                                    vals.append(sqlstr(prof[1]))

                                args.append('profLastName')
                                vals.append(sqlstr(prof[0]))

                            clsDate = cls['date']
                            if clsDate:
                                if clsDate['start_time']:
                                    args.append('classStartTime')
                                    vals.append(sqlstr(clsDate['start_time']))

                                if clsDate['end_time']:
                                    args.append('classEndTime')
                                    vals.append(sqlstr(clsDate['end_time']))

                                if clsDate['weekdays']:
                                    args.append('classWeekdays')
                                    vals.append(sqlstr(clsDate['weekdays']))

                            loc = cls['location']
                            if loc:
                                if loc['building']:
                                    args.append('classBuilding')
                                    vals.append(sqlstr(loc['building']))

                                if loc['room']:
                                    args.append('classRoom')
                                    vals.append(sqlstr(loc['room']))

                        set_courseOffering.add((course_code, term, component))
                        stmts_courseOffering.append(
                            f"INSERT INTO courseOffering ({', '.join(args)}) VALUES ({', '.join(vals)});"
                        )
                    
print('\ndone.')

### Course Prereqs

In [16]:
prereq_endpoint = "https://api.uwaterloo.ca/v2/courses/{}/{}/prerequisites.json?key=e5c5ea41b7715202b2846b04b286f29d&fbclid=IwAR26szkbPavfEMNVDdLaYTznyHkCz8imaQ7PrAd8fUmG2WSjXJiew6kuFsw"
for course in courses:
    subject = course['subject']
    num = course['catalog_number']
    
    course['prerequisites_parsed'] = []
    course_endpoint = prereq_endpoint.format(subject, num)
    r = requests.get(course_endpoint)
    while r.status_code != 200:
        # Test for request error
        print("ERROR!")
        r = requests.get(course_endpoint)
        
    info = r.json()
    status = info['meta']['status']
    
    # Test for API error
    if status == 204:
        # No data returned
        continue
    
    elif status == 200: 
        # Found prereqs
        
        if "data" in info:
            course['prerequisites_parsed'] = info['data']['prerequisites_parsed']
            
            print("Found prereqs for {} {}".format(subject, num))
    else:
        print("ERROR: got status {}".format(info['status']))

Found prereqs for ACTSC 221
Found prereqs for ACTSC 231
Found prereqs for ACTSC 232
Found prereqs for ACTSC 291
Found prereqs for ACTSC 331
Found prereqs for ACTSC 371
Found prereqs for ACTSC 372
Found prereqs for ACTSC 391
Found prereqs for ACTSC 431
Found prereqs for ACTSC 432
Found prereqs for ACTSC 433
Found prereqs for ACTSC 445
Found prereqs for ACTSC 446
Found prereqs for ACTSC 453
Found prereqs for ACTSC 455
Found prereqs for ACTSC 462
Found prereqs for ACTSC 463
Found prereqs for ACTSC 468
Found prereqs for ACTSC 469
Found prereqs for ACTSC 471
Found prereqs for ACTSC 621
Found prereqs for ACTSC 622
Found prereqs for ACTSC 623
Found prereqs for ACTSC 624
Found prereqs for ACTSC 625
Found prereqs for ACTSC 631
Found prereqs for ACTSC 632
Found prereqs for ACTSC 633
Found prereqs for ACTSC 634
Found prereqs for ACTSC 635
Found prereqs for ACTSC 936
Found prereqs for ACTSC 966
Found prereqs for AFM 101
Found prereqs for AFM 102
Found prereqs for AFM 121
Found prereqs for AFM 123


Found prereqs for BIOL 345
Found prereqs for BIOL 346
Found prereqs for BIOL 348L
Found prereqs for BIOL 349
Found prereqs for BIOL 350
Found prereqs for BIOL 351
Found prereqs for BIOL 354
Found prereqs for BIOL 355
Found prereqs for BIOL 359
Found prereqs for BIOL 361
Found prereqs for BIOL 364
Found prereqs for BIOL 365
Found prereqs for BIOL 366
Found prereqs for BIOL 370
Found prereqs for BIOL 371
Found prereqs for BIOL 373
Found prereqs for BIOL 376
Found prereqs for BIOL 377
Found prereqs for BIOL 382
Found prereqs for BIOL 383
Found prereqs for BIOL 403
Found prereqs for BIOL 412
Found prereqs for BIOL 426
Found prereqs for BIOL 428
Found prereqs for BIOL 431
Found prereqs for BIOL 432
Found prereqs for BIOL 433
Found prereqs for BIOL 434
Found prereqs for BIOL 438
Found prereqs for BIOL 439
Found prereqs for BIOL 441
Found prereqs for BIOL 442
Found prereqs for BIOL 443
Found prereqs for BIOL 444
Found prereqs for BIOL 447
Found prereqs for BIOL 448
Found prereqs for BIOL 449


Found prereqs for CO 342
Found prereqs for CO 351
Found prereqs for CO 353
Found prereqs for CO 367
Found prereqs for CO 370
Found prereqs for CO 372
Found prereqs for CO 380
Found prereqs for CO 430
Found prereqs for CO 434
Found prereqs for CO 439
Found prereqs for CO 440
Found prereqs for CO 442
Found prereqs for CO 444
Found prereqs for CO 446
Found prereqs for CO 450
Found prereqs for CO 452
Found prereqs for CO 453
Found prereqs for CO 454
Found prereqs for CO 456
Found prereqs for CO 459
Found prereqs for CO 463
Found prereqs for COGSCI 300
Found prereqs for CO 466
Found prereqs for CO 471
Found prereqs for CO 480
Found prereqs for CO 481
Found prereqs for CO 485
Found prereqs for CO 487
Found prereqs for CO 499
Found prereqs for COMM 101
Found prereqs for COMM 102
Found prereqs for COMM 103
Found prereqs for COMM 321
Found prereqs for COMM 400
Found prereqs for COMM 421
Found prereqs for COMM 431
Found prereqs for COMM 432
Found prereqs for COOP 101
Found prereqs for CROAT 102


Found prereqs for ECE 459
Found prereqs for ECE 462
Found prereqs for ECE 463
Found prereqs for ECE 464
Found prereqs for ECE 467
Found prereqs for ECE 473
Found prereqs for ECE 474
Found prereqs for ECE 475
Found prereqs for ECE 477
Found prereqs for ECE 481
Found prereqs for ECE 492A
Found prereqs for ECE 484
Found prereqs for ECE 486
Found prereqs for ECE 488
Found prereqs for ECE 493
Found prereqs for ECE 498A
Found prereqs for ECE 498B
Found prereqs for ECE 499
Found prereqs for ECE 632
Found prereqs for ECE 667
Found prereqs for ECON 622A
Found prereqs for ECE 6601PD
Found prereqs for ECE 6602PD
Found prereqs for ECE 6603PD
Found prereqs for ECE 6604PD
Found prereqs for ECE 6605PD
Found prereqs for ECE 6606PD
Found prereqs for ECE 6607PD
Found prereqs for ECE 6608PD
Found prereqs for ECE 6609PD
Found prereqs for ECE 6610PD
Found prereqs for ECE 6611PD
Found prereqs for ECE 6612PD
Found prereqs for ECE 6613PD
Found prereqs for ECE 6614PD
Found prereqs for ECE 6615PD
Found prereqs 

Found prereqs for FINE 212
Found prereqs for FINE 213
Found prereqs for FINE 214
Found prereqs for FINE 215
Found prereqs for FINE 220
Found prereqs for FINE 221
Found prereqs for FINE 222
Found prereqs for FINE 223
Found prereqs for FINE 223A
Found prereqs for FINE 224
Found prereqs for FINE 225
Found prereqs for FINE 226A
Found prereqs for FINE 226
Found prereqs for FINE 227
Found prereqs for FINE 228
Found prereqs for FINE 229
Found prereqs for FINE 230
Found prereqs for FINE 231
Found prereqs for FINE 232
Found prereqs for FINE 241
Found prereqs for FINE 242
Found prereqs for FINE 243
Found prereqs for FINE 271
Found prereqs for FINE 272
Found prereqs for FINE 274
Found prereqs for FINE 282
Found prereqs for FINE 293
Found prereqs for FINE 294
Found prereqs for FINE 300
Found prereqs for FINE 301
Found prereqs for FINE 302
Found prereqs for FINE 303
Found prereqs for FINE 304
Found prereqs for FINE 305
Found prereqs for FINE 306
Found prereqs for FINE 319
Found prereqs for FINE 330

...

Found prereqs for SOCWK 300R
Found prereqs for SOCWK 301R
Found prereqs for SOCWK 320R
Found prereqs for SOCWK 321R
Found prereqs for SOCWK 322R
Found prereqs for SOCWK 326R
Found prereqs for SOCWK 355R
Found prereqs for SOCWK 356R
Found prereqs for SOCWK 357R
Found prereqs for SOCWK 365R
Found prereqs for SOCWK 367R
Found prereqs for SOCWK 390A
Found prereqs for SOCWK 390B
Found prereqs for SOCWK 421R
Found prereqs for SOCWK 450R
Found prereqs for SOCWK 490R
Found prereqs for SPAN 102
Found prereqs for SPAN 223W
Found prereqs for SPAN 201A
Found prereqs for SPAN 201B
Found prereqs for SPAN 222W
Found prereqs for SPAN 227
Found prereqs for SPAN 228
Found prereqs for SPAN 301A
Found prereqs for SPAN 301B
Found prereqs for SPAN 305W
Found prereqs for SPAN 326
Found prereqs for SPAN 327W
Found prereqs for SPAN 334
Found prereqs for SPAN 344
Found prereqs for SPAN 350
Found prereqs for SPAN 366
Found prereqs for SPAN 386
Found prereqs for SPAN 387
Found prereqs for SPAN 390
Found prereqs f

Found prereqs for DRAMA 317
Found prereqs for DRAMA 332
Found prereqs for DRAMA 334
Found prereqs for DRAMA 349
Found prereqs for DRAMA 350
Found prereqs for DRAMA 352
Found prereqs for DRAMA 353
Found prereqs for DRAMA 354
Found prereqs for DRAMA 355
Found prereqs for DRAMA 356
Found prereqs for DRAMA 357
Found prereqs for DRAMA 358
Found prereqs for DRAMA 391
Found prereqs for DRAMA 393
Found prereqs for DRAMA 395
Found prereqs for EARTH 205
Found prereqs for EARTH 305
Found prereqs for EARTH 361
Found prereqs for ECE 202
Found prereqs for ECE 204
Found prereqs for ECE 241
Found prereqs for ECE 251
Found prereqs for ECE 261
Found prereqs for ECE 301
Found prereqs for ECE 302
Found prereqs for ECE 324
Found prereqs for ECE 332
Found prereqs for ECE 342
Found prereqs for ECE 355
Found prereqs for ECE 370
Found prereqs for ECE 391
Found prereqs for ECE 401
Found prereqs for ECE 402
Found prereqs for ECE 412
Found prereqs for ECE 428
Found prereqs for ECE 431
Found prereqs for ECE 434
Fo

Found prereqs for EASIA 231R
Found prereqs for EASIA 377R
Found prereqs for ECON 412
Found prereqs for ENGL 294
Found prereqs for ENGL 295
Found prereqs for ERS 374
Found prereqs for ERS 422
Found prereqs for ERS 454
Found prereqs for ERS 473
Found prereqs for FINE 275
Found prereqs for FINE 383
Found prereqs for HIST 377R
Found prereqs for INDEV 476
Found prereqs for INDEV 490A
Found prereqs for INDEV 490B
Found prereqs for INTEG 452A
Found prereqs for INTEG 452B
Found prereqs for JS 338
Found prereqs for LS 240
Found prereqs for LS 425
Found prereqs for MSCI 734
Found prereqs for NE 111
Found prereqs for PHARM 155
Found prereqs for PHARM 379
Found prereqs for PHARM 425
Found prereqs for PHARM 430
Found prereqs for PHARM 440
Found prereqs for PHARM 450
Found prereqs for PHARM 473
Found prereqs for PHARM 474
Found prereqs for PHARM 475
Found prereqs for PHARM 476
Found prereqs for PHARM 496
Found prereqs for PHARM 497
Found prereqs for PHARM 498
Found prereqs for PSCI 497A
Found prereq

Found prereqs for FINE 307
Found prereqs for FINE 308
Found prereqs for FINE 403
Found prereqs for FR 296
Found prereqs for GEOG 325
Found prereqs for GERON 310
Found prereqs for GERON 320
Found prereqs for HIST 421
Found prereqs for HIST 422
Found prereqs for HIST 450
Found prereqs for HLTH 204
Found prereqs for HLTH 230
Found prereqs for HLTH 280
Found prereqs for HLTH 304
Found prereqs for HLTH 320
Found prereqs for HLTH 370
Found prereqs for ENBUS 314
Found prereqs for HLTH 412
Found prereqs for INTEG 420A
Found prereqs for INTEG 420B
Found prereqs for KIN 310
Found prereqs for KIN 332
Found prereqs for MSCI 391
Found prereqs for MSCI 392
Found prereqs for MSCI 491
Found prereqs for MUSIC 333
Found prereqs for MUSIC 335
Found prereqs for NE 345
Found prereqs for NE 350
Found prereqs for NE 381
Found prereqs for NE 450
Found prereqs for PACS 315
Found prereqs for NE 454A
Found prereqs for NE 454B
Found prereqs for NE 454C
Found prereqs for NE 454D
Found prereqs for NE 455A
Found pre

Found prereqs for ARTS 103
Found prereqs for AVIA 315
Found prereqs for BET 580
Found prereqs for CHE 180
Found prereqs for CHE 181
Found prereqs for ECE 101A
Found prereqs for ECE 101B
Found prereqs for ECE 101C
Found prereqs for ECE 101D
Found prereqs for ECE 101E
Found prereqs for ECE 109
Found prereqs for ENBUS 375
Found prereqs for ENBUS 403A
Found prereqs for ENBUS 403B
Found prereqs for ENBUS 475
Found prereqs for ENGL 191
Found prereqs for ENGL 192
Found prereqs for ENVE 115
Found prereqs for ENVE 121
Found prereqs for ENVE 382
Found prereqs for ENVE 392
Found prereqs for ENVE 583
Found prereqs for GEOE 100
Found prereqs for GEOE 115
Found prereqs for GEOE 121
Found prereqs for GEOE 221
Found prereqs for GEOE 223
Found prereqs for GEOE 224
Found prereqs for GEOE 280
Found prereqs for GEOE 354
Found prereqs for GEOE 391
Found prereqs for GEOE 392
Found prereqs for GEOG 315
Found prereqs for GEOG 392
Found prereqs for GEOG 460
Found prereqs for GRK 291
Found prereqs for HLTH 454


In [18]:
# Save them just in case the next step messes them up
import pickle
pickle.dump(courses, open("courses_with_prereqs.pckl", "wb" ))

7449

In [19]:
def formatCourseName(s):
    """
    CS135 -> CS 135
    """
    split_idx = 0
    for i in range(len(s)):
        if s[i].isdigit():
            break
        split_idx += 1
    return s[:split_idx] + " " + s[split_idx:]

assert formatCourseName("CS135") == "CS 135"
assert formatCourseName("MECH223") == "MECH 223"

In [20]:
def flattenORPrereqs(lst):
    # I think I did too much racket programming guys... #foldr
    
    if len(lst) == 0:
        return []
    elif lst[0] == 1:
        return [1] + flattenORPrereqs(lst[1:])
    elif isinstance(lst[0], list) and lst[0][0] == 1:
        return flattenORPrereqs(lst[0][1:]) + flattenORPrereqs(lst[1:])
    elif isinstance(lst[0], list):
        return flattenORPrereqs(lst[0]) + flattenORPrereqs(lst[1:])
    else:
        return [lst[0]] + flattenORPrereqs(lst[1:])
            
    
bad_input = [1, [1, 'AFM372', 'ACTSC391'], ['ACTSC231', 'ACTSC371'], 'ACTSC231BUS']
expected = [1, 'AFM372', 'ACTSC391', 'ACTSC231', 'ACTSC371', 'ACTSC231BUS']
assert flattenORPrereqs(bad_input) == expected

In [28]:
from collections import defaultdict

prereq_mapping = defaultdict(list)

for course in courses:
    course_name = course['subject'] + " " + course['catalog_number']
    prereqs = course['prerequisites_parsed']
    if prereqs is None:
        continue
    if len(prereqs) > 0 and isinstance(prereqs[0], int):
        prereqs = [prereqs]
    
    for pre in prereqs:
        if isinstance(pre, list) and len(pre) > 1:
            pre = flattenORPrereqs(pre)
            pre[0] = 1 # Some, like FRENCH, will have 2. Just make it one for simplicity...
            # Make a course group
            course_group = []
            for c in pre[1:]:
                g = formatCourseName(c)
                course_group.append(g)
            course_group = list(map(formatCourseName, pre[1:]))
            course_group = list(filter(lambda c: c in set_course, course_group))
            if len(course_group) == 0:
                continue
            prereq_mapping[course_name].append(course_group)
        else:
            # A single required course
            try:
                if pre in set_course:
                    prereq_mapping[course_name].append([pre])
            except:
                # Probably an error with the UW API
                continue
        
        ### DEBUG ###
        #if len(prereq_mapping[course_id]) > 0:
        #    print(prereq_mapping[course_id][-1])

In [30]:
### At this point each course has its prereqs parsed. (multiple coureses in a course group -> OR)
prereq_mapping

defaultdict(list,
            {'ACTSC 231': [['MATH 137', 'MATH 147'],
              ['STAT 220', 'STAT 230', 'STAT 240'],
              ['ACTSC 232']],
             'ACTSC 232': [['STAT 230', 'STAT 240'], ['ACTSC 331']],
             'ACTSC 291': [['MATH 136', 'MATH 146'], ['MATH 138', 'MATH 148']],
             'ACTSC 372': [['MATH 207', 'MATH 237', 'MATH 247']],
             'ACTSC 391': [['AFM 272', 'ACTSC 291']],
             'ACTSC 445': [['AFM 372', 'ACTSC 391', 'ACTSC 231', 'ACTSC 371'],
              ['STAT 330', 'STAT 333', 'STAT 334']],
             'ACTSC 446': [['AFM 372', 'ACTSC 391', 'ACTSC 231'],
              ['STAT 333', 'STAT 334']],
             'ACTSC 462': [['AFM 272', 'ACTSC 291', 'ACTSC 231']],
             'ACTSC 463': [['AFM 272', 'ACTSC 291', 'ACTSC 231'],
              ['STAT 331', 'STAT 371', 'STAT 373']],
             'ACTSC 471': [['AFM 372', 'ACTSC 391', 'ACTSC 372']],
             'ACTSC 936': [['STAT 431', 'STAT 831']],
             'AFM 102': [['AFM 1

In [31]:
# This is just a sanity check to make sure all the courses and their prereqs are in `set_course`
for k, v in prereq_mapping.items():
    if k not in set_course:
        raise Exception("{} is not in `set_course`, make some deletions!")
        break
    for l in v:
        for p in l:
            if p not in set_course:
                raise Exception("{} is not in `set_course`, make some deletions!".format(p))

In [32]:
# Create a course group
courseGroupQuery = """INSERT INTO courseGroup (quantity) VALUES ({});"""
courseGroupMemberQuery = """
WITH groupID as(
    SELECT currval(pg_get_serial_sequence('courseGroup', 'groupid')) as v
)
INSERT INTO courseGroupMember (courseCode, courseGroupID) VALUES ('{}', (SELECT v from groupID));"""

# Add a course group as a prereq
addPrereqQuery = """
WITH groupID as(
    SELECT currval(pg_get_serial_sequence('courseGroup', 'groupid')) as v
)
INSERT INTO prerequisite (courseCode, prereqCourseGroupID) VALUES ('{}', (SELECT v from groupID));
"""

In [33]:
stmts_prerequisite = []

for course, prereqs in prereq_mapping.items():
    
    # Comments for the insert file
    stmts_prerequisite.append( "-" * 100)
    stmts_prerequisite.append( "--- {} --> {}".format(course, prereqs))
    stmts_prerequisite.append( "-" * 100)
    
    for or_prereq_lst in prereqs:
 
        stmts_prerequisite.append( courseGroupQuery.format(len(or_prereq_lst)) )
            
        for p in or_prereq_lst:
            stmts_prerequisite.append( courseGroupMemberQuery.format(p) )
        
        stmts_prerequisite.append( addPrereqQuery.format(course))

### Output to Files

In [20]:
with open('insert_1_faculty.sql', 'w+') as f:
    for line in stmts_faculty:
        f.write(line + '\n')

In [21]:
with open('insert_2_subject.sql', 'w+') as f:
    for line in stmts_subject:
        f.write(line + '\n')

In [22]:
with open('insert_3_course.sql', 'w+') as f:
    for line in stmts_course:
        f.write(line + '\n')

In [23]:
with open('insert_4_term.sql', 'w+') as f:
    for line in stmts_term:
        f.write(line + '\n')

In [36]:
with open('insert_5_courseOffering.sql', 'w+') as f:
    for line in stmts_courseOffering:
        f.write(line + '\n')

In [35]:
with open('insert_6_prerequisite.sql', 'w') as f:
    for query in stmts_prerequisite:
        f.write(query + "\n")

### Export Course.txt

In [121]:
with open('course.txt', 'w+') as f:
    for line in set_course:
        f.write(line + '\n')